This notebook will serve as a testbed for custom shape-targeting forward and backward development and testing.

Every cell done below would able to run independently after first import

In [1]:
# First, add a custom function to deformable.h and deformable.cpp that make sure compiles and can modify data
import sys
sys.path.append('../')
import os

from pathlib import Path
import time
import numpy as np
import scipy.optimize
import pickle
import matplotlib.pyplot as plt

from py_diff_pd.common.common import ndarray, create_folder
from py_diff_pd.common.common import print_info, print_ok, print_error, print_warning
from py_diff_pd.common.grad_check import check_gradients
from py_diff_pd.common.display import export_gif
from py_diff_pd.core.py_diff_pd_core import StdRealVector
from py_diff_pd.env.soft_starfish_env_3d import SoftStarfishEnv3d
from py_diff_pd.common.project_path import root_path
from py_diff_pd.core.py_diff_pd_core import HexMesh3d, HexDeformable, StdRealVector
import py_diff_pd.common.hex_mesh as hex

ShapeTarget Compile test

In [2]:
# initailize an array of 0, len 100
x = np.zeros(100)
x = StdRealVector(x)
y = np.zeros(100)
y = StdRealVector(y)

deformable = HexDeformable()

# const q, v, actuation, dt, pd_options, 
# writable q_next, v_next 
# Assume 3000 elements, 5000 vertices 
dt = 0.01
dof  =200
options = {
    'max_pd_iter': 1000,
    'thread_ct': 20,
    'abs_tol': 1e-6,
    'rel_tol': 1e-6,
    'verbose': 0,
    'use_bfgs': 1,
    'bfgs_history_size': 10,
    'max_ls_iter': 10,
    
}
q, v, act =  StdRealVector(dof), StdRealVector(dof), StdRealVector(dof)
q_next, v_next = StdRealVector(dof), StdRealVector(dof)
deformable.PyShapeTargetingForward( q, v, act, dt, options, q_next, v_next)
q_next = np.array(q_next)
q_next



array([], dtype=float64)

Prepare for testing the forward pass:
* We can either add f_ext to it and see if it behaves just like corotated （good, but no enough）
* we use the target shape from starfish testing to see if it can produce the same result. Would also be interesting to see if it requires some iterations for sure.

In [ ]:
# Load the json output from soft_starfish.ipynb, 
# which output the ideal pos for each hex verts.
# Then we can compute the Deformation gradient, and take the middle as the target
